In [1]:
import serial
ser = serial.Serial('/dev/ttyUSB1', 115200, timeout=60*60)

In [ ]:
while x := ser.read(1):
    try:
        print(x.decode('utf-8'), end='')
    except:
        print(x, end='')

In [3]:
import pandas as pd
from datetime import datetime
# Headers and datatypes for pandas dframe
header_raw = (r'^ID;Timestamp;GyroscopeX;GyroscopeY;GyroscopeZ;AccelerometerX;AccelerometerY;',
           'AccelerometerZ;Barometer;Thermometer;ThermometerStupido;Voltage$')

headers = [
    "ID", "Timestamp", "GyroscopeX", "GyroscopeY", "GyroscopeZ", "AccelerometerX", "AccelerometerY",
    "AccelerometerZ", "Barometer", "Thermometer", "ThermometerStupido", "Voltage",
]

datatypes = [int, int, float, float, float, float, float, float, int, float, float, float]


In [5]:
def init_dataframe():
    return pd.DataFrame({}, columns=headers).set_index('ID')

def dump_dataframe(df):
    df.to_csv(f'LOG_{datetime.now().isoformat()}.csv')

def is_csv_header(line):
    # Simple heuristic to check if a line is a CSV header
    return bool(re.match(header_raw, line))

def is_csv_row(line):
    # Simple heuristic to check if a line could be a CSV row
    return bool(re.match(r'^-?\d+(\.\d+)?(;-?\d+(\.\d+)?)*$', line))

def append_csv_row(row, df):
    values = row.split(';')
    df.append({header: dt(val) for header, dt, val in zip(headers, datatypes, values)})

def process_line(line, df):
    if is_csv_header(line):
        dump_dataframe(df)
    elif is_csv_row(line):
        append_csv_row(line, df)
        

In [3]:
import sys
import re

import pandas as pd
import serial

# Expected indexes of gyro and acc
gyro = [2, 3, 4]
aclr = [5, 6, 7]

# Activates when CSV header is read

device = '/dev/ttyUSB1'
ser = serial.Serial(device, 115200, timeout=60*60)
line = []

df = init_dataframe()

while x := ser.read(1):
    try:
        parsed_x = x.decode("UTF-8")
        if parsed_x == '\n':
            process_line(line)
        else:
            line = line + parsed_x
    except:
        print(x, end='')


b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'\r'b'\n'b'I'b't'b'e'b'r'b'a'b't'b'i'b'o'b'n'b':'b' 'b'4'b'3'b'9'b'\r'b'\n'b'1'b')'b' 'b'b'b'u'b'f'b'f'b'e'b'r'b'e'b'd'b' 'b'm'b'e'b's'b's'b'a'b'g'b'e'b's'b':'b'\r'b'\n'b'2'b')'b' 'b'S'b'e'b'n'b'd'b'i'b'n'b'g'b':'b' 'b'\r'b'\n'b'S'b'e'b'n'b't'b' 'b'O'b'K'b' 'b'.'b'.'b'.'b'\r'b'\n'b'A'b'c'b'k'b' 'b'f'b'o'b'r'b' 'b'l'b'a'b's'b't'b' 'b'm'b'e'b's'b's'b'a'b'g'b'e'b':'b' 'b'4'b'3'b'9'b'\r'b'\n'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'+'b'\r'b'\n'b'I'b't'b'e'b'r'b'a'b't'b'i'b'o'b'n'b':'b' 'b'4'b'4'b'0'b'\r'b'\n'b'1'b')'b' 'b'b'b'u'b'f'b'f'b'e'b'r'b'e'b'd'b' 'b'm'b'e'b's'b's'b'a'b'g'b'e'b's'b':'b'\r'b'\n'b'2'b')'b' 'b'S'b'e'b'n'b'd'b'i'b'n'b'g'b':'b' 'b'\r'b'\n'b'S'b'e'b'n'b't'b' 'b'O'b'K'b' 'b'.'b'.'b'.'b'\r'b'\n'b'A'b'

KeyboardInterrupt: 